# MAVIS Astrometric Simulator
## E2E Version
In this notebook, we simulate a MAVIS image of the NGC 3201 catalogue (truncated to 4000 stars) using a grid of 11x11 E2E simulated PSFs.

# Imports

In [ ]:
# Standard
import numpy as np
import matplotlib.pyplot as plt

# Astropy
from astropy.io import fits, ascii

import input_parameters as input_par

from mavisim.generate_image import ImageGenerator
from mavisim.source import Source
from mavisim.input_coo import InputCoo
from mavisim.noise import add_all_noise

# Create the source object (to populate the image)

In [ ]:
exp_time = 30 # seconds
source = Source(input_par, exp_time, static_dist=True)
source.build_source()

### Create the input catalogue to compare with DAOPhot

In [ ]:
input_coo = InputCoo(input_par, source).main()

# Create the Noise-Free Image with E2E PSF
- Initialise ImageGenerator object
- Generate image,
- Rebin to desired pixel scale.

In [ ]:
# The inputs to image generator aren't very user-friendly regrettably, 
# something to be worked on in future versions.
image_gen = ImageGenerator(array_width_pix=12800, pixsize=3.75e-3, source=source,
                            psfs_file="data/e2e_psfs_100s_lqg_optsquare.fits", gauss_width_pix=34)
image_gen.main()

image_final = image_gen.get_rebinned_cropped(rebin_factor=2,cropped_width_as=30.0)

In [ ]:
print (image_final.shape)

In [ ]:
# plot final image:
extent = 42*np.array([-0.5,0.5,-0.5,0.5])
plt.figure(figsize=[10,10])
plt.imshow(image_final, extent=extent)
plt.colorbar()
plt.clim([8,15])

### Add all noise

In [ ]:
# Add sky background,
#shot noise, read noise and convert from electrons to ADU
image_allnoise = add_all_noise(input_par, image_final, source.exp_time)

# Save the final image and input catalogue

In [ ]:
image_final_noise = np.array(image_allnoise, dtype="float32")
hdu = fits.PrimaryHDU(image_final_noise)
hdul = fits.HDUList([hdu])
hdul.writeto("output_image_001.fits", overwrite=True)
ascii.write(input_coo, 'output_coo_001.all', overwrite=True)